# Digital cut-up and color usage analysis

###  <span style="color:green;"> of the original poem `Soap Bubbles` by Hermann Hesse (1877) </span>

In [123]:
import random
import spacy

In [124]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 4)

In [125]:
from simpleneighbors import SimpleNeighbors

In [126]:
nlp = spacy.load('en_core_web_md')

In [127]:
newversion = nlp(open("Soap Bubbles.txt").read())

In [128]:
print(newversion)

From years of study and of contemplation
An old man brews a work of clarity,
A gay and involuted dissertation
Discoursing on sweet wisdom playfully.

An eager student bent on storming heights
Has delved in archives and in libraries,
But adds the touch of genius when he writes
A first book full of deepest subtleties.

A boy, with bowl and straw, sits and blows,
Filling with breath the bubbles from the bowl.
Each praises like a hymn, and each one glows;
Into the filmy beads he blows his soul.

Old man, student, boy, all these three
Out of the Maya-foam of the universe
Create illusions. None is better or worse.
But in each of them the Light of Eternity
Sees its reflection, and burns more joyfully.



In [129]:
def vec(s):
    return nlp.vocab[s].vector

In [130]:
!curl -L -O https://raw.githubusercontent.com/aparrish/wordfreq-en-25000/main/wordfreq-en-25000-log.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  898k  100  898k    0     0  4383k      0 --:--:-- --:--:-- --:--:-- 4447k


In [131]:
import json
prob_lookup = dict(json.load(open("./wordfreq-en-25000-log.json")))

In [132]:
lookup = SimpleNeighbors(300)
for word in prob_lookup.keys():
    if nlp.vocab[word].has_vector:
        lookup.add_one(word, vec(word))
lookup.build()

In [133]:
output = []
for word in newversion:
    if word.is_alpha and word.pos_ in ('NOUN', 'ADJ','ADV'):
        new_word = random.choice(lookup.nearest(word.vector, 10))
        output.append(new_word)
    else:
        output.append(word.text)
    output.append(word.whitespace_)
print(''.join(output))

From age of studies and of florence
An woman housewife brews a internship of confidence,
A lesbians and involuted exposition
Discoursing on fudge divine charm.

An annoyed academic bent on storming tension
Has delved in archival and in library,
But adds the comfortably of marvellous when he writes
A firstly brochure superb of deepest misconceptions.

A lady, with cup and terry, sits and blows,
Filling with gently the droplets from the jar.
Each igor like a rhymes, and each one glows;
Into the films bead he blows his abode.

prairie man, students, boy, all these three
Out of the Maya-poly of the universal
Create essence. nothin is harden or shameful.
But in each of them the Light of illusion
Sees its liberalization, and burns rather lovely.



In [136]:
chunk_lookup = SimpleNeighbors(300)
for chunk in newversion.noun_chunks:
    chunk_text = chunk.text.replace("\n", " ")
    if chunk_text not in chunk_lookup.corpus:
        chunk_lookup.add_one(chunk_text, chunk.vector)
chunk_lookup.build()

In [137]:
chunk_lookup.nearest(nlp("soap bubbles").vector)

['straw',
 'bowl',
 'the bubbles',
 'the filmy beads',
 'sweet wisdom',
 'breath',
 'the bowl',
 'them',
 'the Maya-foam',
 'the touch',
 'a work',
 'each']

In [138]:
chunk_lookup.nearest(nlp("sweet").vector)

['sweet wisdom',
 'straw',
 'bowl',
 'breath',
 'a hymn',
 'genius',
 'them',
 'his soul',
 'A boy',
 'clarity',
 'deepest subtleties',
 'Old man']

In [139]:
chunk_lookup.nearest(nlp("deepest").vector)

['deepest subtleties',
 'sweet wisdom',
 'the universe Create illusions',
 'the touch',
 'his soul',
 'clarity',
 'the bubbles',
 'heights',
 'breath',
 'Eternity',
 'its reflection',
 'them']

In [140]:
output = []
for word in newversion:
    if word.is_alpha and word.pos_ == 'NOUN':
        new_word = random.choice(chunk_lookup.nearest(word.vector,4))
        output.append(new_word)
    elif word.is_alpha and word.tag_ == 'ADJ':
        new_word = random.choice(adj_lookup.nearest(word.vector, 4))
        output.append(new_word)
    else:
        output.append(word.text)
    output.append(word.whitespace_)
print(''.join(output))

From archives of libraries and of deepest subtleties
An old Old man brews a student of clarity,
A gay and involuted study
Discoursing on sweet deepest subtleties playfully.

An eager study bent on storming deepest subtleties
Has delved in contemplation and in the universe Create illusions,
But adds the sweet wisdom of sweet wisdom when he writes
A first libraries full of deepest deepest subtleties.

A A boy, with straw and bowl, sits and blows,
Filling with breath the straw from the the bowl.
Each clarity like a Old man, and each one glows;
Into the filmy sweet wisdom he blows his his soul.

Old An old man, libraries, boy, all these three
Out of the Maya-straw of the universe
Create deepest subtleties. deepest subtleties is better or worse.
But in each of them the Light of Eternity
Sees its deepest subtleties, and burns more joyfully.



In [141]:
sentence_lookup.nearest(nlp("Read to me.").vector)

['Each praises like a hymn, and each one glows; Into the filmy beads he blows his soul.  ',
 'None is better or worse. ',
 'But in each of them the Light of Eternity Sees its reflection, and burns more joyfully. ',
 'An eager student bent on storming heights Has delved in archives and in libraries, But adds the touch of genius when he writes A first book full of deepest subtleties.  ',
 'A boy, with bowl and straw, sits and blows, Filling with breath the bubbles from the bowl. ',
 'Old man, student, boy, all these three Out of the Maya-foam of the universe Create illusions.',
 'From years of study and of contemplation An old man brews a work of clarity, A gay and involuted dissertation Discoursing on sweet wisdom playfully.  ']

## Digital color analysis

In [143]:
color_data = json.loads(open("xkcd.json").read())

In [144]:
from numpy.linalg import norm

def distance(a,b):
    return norm(a-b)

In [145]:
def hex_to_int(s):
    s = s.lstrip("#")
    return np.array([int(s[:2], 16), int(s[2:4], 16), int(s[4:6], 16)])

In [146]:
colors = dict()
for item in color_data['colors']:
    colors[item['color']]= hex_to_int(item['hex'])


In [147]:
def meanv(vecs):
    total = np.sum(vecs, axis=0)
    return total / len(vecs)

In [156]:
color_lookup = SimpleNeighbors(3,'euclidean')
for name,vec in colors.items():
    color_lookup.add_one(name,vec)
color_lookup.build()

In [160]:
drac_colors = [colors[word.lower_] for word in newversion if word.lower_ in colors]
avg_colors = meanv(drac_colors)
print(avg_colors)

[252. 246. 121.]


In [159]:
color_lookup.nearest(avg_colors,5)

['straw', 'custard', 'light yellow', 'butter yellow', 'sandy yellow']

### The digital color of `Soap Bubbles`:

<img src="color1.png" width="450" height="450">

## - The End -